In [1]:
from torch import empty
import math
from NN_Modules import LossMSE, ReLU, Tanh, Sigmoid, FCC, Sequential, modelTester, modelTrainer

In [2]:
#torch.set_grad_enabled(False)
train_size = 1000
test_size = 200
valid_size = 100
train_input = empty(train_size, 2).uniform_(0,1)
train_target = train_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

valid_input = empty(valid_size, 2).uniform_(0,1)
valid_target = valid_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

test_input = empty(test_size, 2).uniform_(0,1)
test_target = test_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)


train_noise = empty(train_size, 2).normal_(0,0.05) 
valid_noise = empty(valid_size, 2).normal_(0,0.05) 
test_noise = empty(test_size, 2).normal_(0,0.05)

train_input += train_noise
valid_input += valid_noise
test_input += test_noise


seq = Sequential(["FCC","Tanh","FCC"],[[2,8],[], [8,1]],"MSE",momentum = 0.2)
trained_model ,_ ,_  = modelTrainer(seq,1000, train_input,train_target,valid_input,valid_target, 20, 10)



print(modelTester(trained_model, test_input,test_target,10) )




0.92


In [11]:
def in_circle(dataset):
    dataset = dataset.add(-0.5) #Center the points
    target = empty(dataset.shape[0], 2)
    target[:,0] = dataset.pow(2).sum(1) <= 1 / (2 * math.pi)
    target[:,1] =  (target[:,0] != 1)
    return target

train_size = 1000
test_size = 1000
train_input = empty(train_size, 2).uniform_(0,1)
train_target = in_circle(train_input)

test_input = empty(test_size, 2).uniform_(0,1)
test_target = in_circle(test_input)


model = Sequential(["FCC","Tanh", "FCC","ReLU","FCC","Tanh", "FCC"],[[2,25],[], [25,25],[], [25,25],[], [25,2]],"MSE",momentum = 0.1)
trained_model ,loss, train_acc  = modelTrainer(model = model , num_epoch = 300, train_input = train_input,train_target = train_target, train_batch = 20)
test_acc = modelTester(trained_model, test_input,test_target,10)
print("Final Test accuracy: ", test_acc)
print("Final Train accuracy: ", train_acc[0][-1])

Final Test accuracy:  0.958
Final Train accuracy:  0.969


### Benchmarking

In [12]:
from torch import optim, Tensor, nn
from torch import Tensor
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.set_grad_enabled(True)

def create_model():
    return nn.Sequential(
        nn.Linear(2, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
#         nn.ReLU(),
)

def train_model(model, train_input, train_target):
    loss_track = []
    loss_func = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.1)
    mini_batch = 200
    # Iterate through epochs
    for i in range(1000):
        #Iterate t hrough mini batches
        for b in range(0, train_input.size(0), mini_batch):
            # Forward pass selecting the corresponding minibacth
            output = model(train_input.narrow(0, b, mini_batch))
            # Calculate loss
            loss = loss_func(output.squeeze(), train_target.narrow(0, b, mini_batch))
#             print("Loss:", loss.item())
            # Reset zero grad
            model.zero_grad()
            # Backward pass
            loss.backward()
            #Update step
            optimizer.step()
        loss_track.append(loss)
    plt.plot(loss_track)
    
def compute_nb_errors(model, data_input, data_target):
        output = model(data_input)
        comp = output.argmax(dim = 1) != data_target.argmax(dim = 1)
        nb_errors = torch.where(comp)[0].size()[0]
        error_rate = nb_errors/data_input.size(0)
        print(f'There were a total of {nb_errors} errors, or {error_rate} of the testing set.')
        
        
model = create_model()
train_model(model, train_input, train_target)
# compute_nb_errors(model, test_input, test_target)

E:\anaconda\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([200, 2])) that is different to the input size (torch.Size([200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (200) must match the size of tensor b (2) at non-singleton dimension 1